# Going deep into Human Activity Recognition

**Elia Bonetto, Filippo Rigotto.**

Department of Information Engineering, University of Padova, Italy.

Human Data Analytics, a.y. 2018/2019

## Part 2 - Training of DL models

In [0]:
!nvidia-smi # watch for T4

In [0]:
from IPython.display import Image, clear_output
import os
from google.colab import drive
drive.mount('/content/drive/')
clear_output()
os.chdir("/content/drive/My Drive/hda-project")
#!ls

In [0]:
!pip install telepot
clear_output()

from pprint import pprint
import json
from datetime import datetime
import pytz

import math
import h5py
import numpy as np
import scipy as sp
import scipy.io

import pandas as pd
pd.set_option('display.precision',3)
pd.set_option('display.float_format', '{:0.3f}'.format)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Dropout
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, LSTM, GRU, Bidirectional
from tensorflow.keras.layers import Bidirectional, TimeDistributed, RepeatVector, UpSampling1D, UpSampling2D, ZeroPadding2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical, plot_model
from tg_callback import TelegramCallback

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')

## Data loading

Start from previously preprocessed data, altrady splitted in train and test parts.

In [0]:
map_decode = {
    0: 'running',
    1: 'walking',
    2: 'jumping',
    3: 'standing',
    4: 'sitting',
    5: 'lying',
    6: 'falling'
}
num_classes = len(map_decode)

In [0]:
# this dataset will be used for all preliminary tests below
with h5py.File('dataset/ARS-train-test-body-framed-aug-onlytrain-rot-per-norm.h5','r') as h5f:
    X_train = h5f['X_train'][:] # IMU data w.r.t body frame
    X_test  = h5f['X_test'][:]  # activities (labels)
    Y_train = h5f['Y_train'][:]
    Y_test  = h5f['Y_test'][:]

num_data = len(X_train)
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("Y_test shape:  " + str(Y_test.shape))

# categorical structures are needed for the loss function to work properly
# original test classes are needed for prediction steps
Y_train_orig = Y_train.copy()
Y_test_orig  = Y_test.copy()
Y_train = to_categorical(Y_train, num_classes=num_classes, dtype=np.uint8)
Y_test  = to_categorical(Y_test,  num_classes=num_classes, dtype=np.uint8)

## Training and evaluation

Precision, recall and F1 score are implemented referring to Tensorflow backend and are passed as custom metrics to track during training and evaluation of models.

Other than monitoring accuracy, we further define three metrics:

$m_2 = \frac{accuracy}{loss} \qquad m_3 = accuracy+precision+recall \qquad m_4 = \frac{accuracy+precision+recall}{loss}$

All of these are referred to test-set values, and models are saved when these quantities reach a new maximum.

The `run_model` function takes care of bootstrap, training and evaluation processes for a given Keras model and configuration.

In [0]:
def recall(y_true, y_pred):
    """Recall metric, batch-wise average."""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric, batch-wise average."""
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    """F1 score, based on precision and recall metrics."""
    prc = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prc*rec)/(prc+rec+K.epsilon()))

def save_custom(epoch, logs):
    """Callback method to save models on metrics change."""
    global output_dir
    global max_accu
    global min_loss
    global max_accu_over_loss
    global max_sum_apr
    global max_sum_aprol
    global epochs
    global is_ae
    
    vaccu = logs['val_acc']
    vloss = logs['val_loss']
    if not is_ae:
        vprec = logs['val_precision']
        vrec  = logs['val_recall']

    if not is_ae:
        if (vaccu + vprec + vrec) > max_sum_apr:
            max_sum_apr = (vaccu + vprec + vrec)
            model.save(os.path.join(out_folder, 'model-best-apr.h5'))
            epochs['apr'] = epoch # save to log
        
        if (vaccu + vprec + vrec) / vloss > max_sum_aprol:
            max_sum_aprol = (vaccu + vprec + vrec) / vloss
            model.save(os.path.join(out_folder, 'model-best-aprol.h5'))
            epochs['aprol'] = epoch

    if vaccu < max_accu and vaccu > max_accu - 0.01 and vloss < min_loss:
        model.save(os.path.join(out_folder, 'model-best-al.h5'))
        epochs['al'] = epoch
    
    if vaccu / vloss > max_accu_over_loss:
        max_accu_over_loss = vaccu/vloss
        model.save(os.path.join(out_folder, 'model-best-aol.h5'))
        epochs['aol'] = epoch

    if vaccu > max_accu:
        max_accu = vaccu
        model.save(os.path.join(out_folder, 'model-best-a.h5'))
        epochs['a'] = epoch
    if vloss < min_loss:
        min_loss = vloss

In [0]:
def run_model(model, config, 
              x_train = X_train, y_train = Y_train, 
              x_test = X_test, y_test = Y_test, y_test_orig = Y_test_orig, is_autoencoder=False, folder_suffix=None):
    """Generic method to build a model, train and evaluate performance."""

    global out_folder
    out_folder = os.path.join('output', datetime.now(pytz.timezone('Europe/Rome')).strftime('%y%m%d-%H%M%S')+ (('_'+folder_suffix) if folder_suffix else '') +'_'+model.name)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    
    # print and save model summary
    print('Summary')
    model.summary(line_length=100)
    with open(os.path.join(out_folder, 'summary.txt'),'w') as sfile:
        model.summary(line_length=100, print_fn=lambda x: sfile.write(x+'\n'))
    plot_model(model, to_file=os.path.join(out_folder, 'model.png'), show_shapes=True)

    # save config
    with open(os.path.join(out_folder, 'config.json'),'w') as cfile:
        json.dump(config, cfile, indent=2)


    if 'lr' not in config:
        # use default values: ass the string
        opt = config['optimizer']
    else:
        # setup optimizer with supplied parameters
        if 'sgdm' in config['optimizer']:
            opt = SGD(lr=config['lr'], momentum=config['momentum'], decay=config['decay'])

        elif 'sgd' in config['optimizer']:
            if 'decay' in config:
                opt = SGD(lr=config['lr'], decay=config['decay'])
            else:
                opt = SGD(lr=config['lr'])

        elif 'adam' in config['optimizer']:
            if 'decay' in config:
                opt = Adam(lr=config['lr'], decay=config['decay'])
            else:
                opt = Adam(lr=config['lr'])

        elif 'rmsprop' in config['optimizer']:
            if 'decay' in config:
                opt = RMSprop(lr=config['lr'], decay=config['decay'])
            else:
                opt = RMSprop(lr=config['lr'])


    # compile model
    if is_autoencoder:
        model.compile(optimizer=opt,
                      loss=config['loss'],
                      metrics=['accuracy'])
    else:
        model.compile(optimizer=opt,
                  loss=config['loss'],
                  metrics=['accuracy', precision, recall, f1])

    # add requested callbacks for model, starting from checkpointing
    callbacks = [ LambdaCallback(on_epoch_end=save_custom) ]

    if config['lr_step'] > 0:
        # halves lr every lr_step epochs (starting lr = 0.01)
        callbacks.append(LearningRateScheduler(
            lambda epoch: 0.01 * math.pow(0.5, math.floor((1+epoch)/config['lr_step'])),
            verbose=1))
        
    if config['early_stop'] > 0:
        # stop if val_loss does has not diminished after num epochs
        callbacks.append(EarlyStopping(patience=config['early_stop']))
        
    if config['tg']:
        # telegram notification when training stops
        callbacks.append(TelegramCallback(name=model.name))
    
    # train model, save final state and history
    print('\nTraining')

    # by default, use test set also as validation set
    x_val = x_test
    y_val = y_test    
    if 'use_validation' in config and config['use_validation']:
        # generate validation set excluding it from the training set
        x_train, x_val, y_train, y_val = \
            train_test_split(x_train, y_train, test_size=0.2, random_state=1, stratify=y_train)     
    
    # global variables to save model in LambdaCallback
    global max_accu
    global min_loss
    global max_accu_over_loss
    global max_sum_apr
    global max_sum_aprol
    global epochs
    global is_ae
    is_ae = is_autoencoder
    max_accu = 0.0
    min_loss = 1e10
    max_accu_over_loss = 0.0
    max_sum_apr = 0.0
    max_sum_aprol = 0.0
    if is_ae:
        epochs = {'a':0, 'al':0, 'aol':0}
    else:
        epochs = {'a':0, 'al':0, 'aol':0, 'apr':0, 'aprol':0}
    
    history = model.fit(x=x_train, y=y_train,
                        shuffle=config['shuffle'],
                        epochs=config['epochs'],
                        batch_size=config['batch_size'],
                        callbacks=callbacks, # if len(callbacks) > 0 else None,
                        validation_data=(x_val,y_val))
    
    model.save(os.path.join(out_folder, 'model-final.h5'))
    
    with open(os.path.join(out_folder, 'history.json'),'w') as hfile:
        hpd = pd.DataFrame(history.history)
        json.dump(json.loads(hpd.to_json()), hfile, indent=2)

    with open(os.path.join(out_folder, 'epochs.json'),'w') as hfile:
        json.dump(epochs, hfile, indent=2)

    # plot and save loss, accuracy and metrics (precision, recall, f1)
    print('\nLoss, accuracy and metrics plots')
    plt.figure()
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-loss')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')
    plt.close()

    plt.figure()
    plt.plot(history.history['acc'], label='Training')
    plt.plot(history.history['val_acc'], label='Validation')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim((0,1))
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-accuracy')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')
    plt.close()

    if not is_ae:
        plt.figure()
        plt.plot(history.history['precision'], label='Precision Tr')
        plt.plot(history.history['val_precision'], label='Precision Val')
        plt.plot(history.history['recall'], label='Recall Tr')
        plt.plot(history.history['val_recall'], label='Recall Val')
        plt.plot(history.history['f1'], label='F1 Tr')
        plt.plot(history.history['val_f1'], label='F1 Val')
        plt.legend()
        plt.xlabel('Epoch')
        plt.ylabel('Metrics')
        plt.tight_layout()
        fname = os.path.join(out_folder, 'plot-metrics')
        plt.savefig(fname+'.png')
        plt.savefig(fname+'.pdf', format='pdf')
        plt.close()

    # evaluate models, save results
    if 'metrics_to_test' in config:
        suffixes = ['final'] + config['metric_to_test']
    else:
        suffixes = ['final','best-a','best-al','best-aol','best-apr','best-aprol']

    for model_suffix in suffixes:
        model_name = os.path.join(out_folder, 'model-{}.h5'.format(model_suffix))
        if not os.path.exists(model_name): continue
        
        if is_ae:
            model = load_model(model_name)
            print(f"\nEvaluation of {model_suffix}")
            metrics = model.evaluate(x=x_test, y=y_test)
            metrics = dict(zip(model.metrics_names, metrics)) # build a dict adding names
            metrics['name'] = model.name
            metrics['type'] = model_suffix
            
            # conversion to pure python float before saving to json
            for item in metrics:
                if type(metrics[item]) == np.float64 or type(metrics[item]) == np.float32:
                    metrics[item] = float(metrics[item])
            with open(os.path.join(out_folder, f"evaluation-{model_suffix}.json"),'w') as efile:
                json.dump(metrics, efile, indent=2)
            
        else:        
            model = load_model(model_name, custom_objects={'precision': precision, 'recall': recall, 'f1': f1})

            print(f"\nEvaluation of {model_suffix}")
            metrics = model.evaluate(x=x_test, y=y_test)
            metrics = dict(zip(model.metrics_names, metrics)) # build a dict adding names
            metrics['name'] = model.name
            metrics['type'] = model_suffix

            # get predictions
            preds = model.predict(x=x_test)
            y_pred = np.argmax(preds, axis=1)

            classes_num = list(map(str,range(num_classes))) # classes list as str integers
            classes = list(map_decode.values())
            metrics['classes'] = classes

            # build per-class metrics and confusion matrix
            cr = classification_report(y_test_orig, y_pred, output_dict=True)

            cm = confusion_matrix(y_test_orig, y_pred)
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normalization

            acc_class = [cm[i,i] for i in range(num_classes)]
            prc_class = [cr[cl]['precision'] for cl in cr if cl in classes_num] # exclude avgs
            rec_class = [cr[cl]['recall']    for cl in cr if cl in classes_num]
            f1_class  = [cr[cl]['f1-score']  for cl in cr if cl in classes_num]

            metrics['acc-class'] = acc_class
            metrics['precision-class'] = prc_class
            metrics['recall-class'] = rec_class
            metrics['f1-class'] = f1_class
            metrics['averages'] = cr['macro avg']
            metrics['weighted-averages'] = cr['weighted avg']
            del metrics['averages']['support']
            del metrics['weighted-averages']['support']
            print()
            pprint(metrics)

            # conversion to pure python float before saving to json
            for item in metrics:
                if type(metrics[item]) == np.float64 or type(metrics[item]) == np.float32:
                    metrics[item] = float(metrics[item])

            # save evaluation dict, confusion matrix and its plot
            with open(os.path.join(out_folder, f"evaluation-{model_suffix}.json"),'w') as efile:
                json.dump(metrics, efile, indent=2)

            np.save(os.path.join(out_folder, f"confusion-{model_suffix}.npy"), cm)

            plt.figure()
            sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=classes, yticklabels=classes)
            plt.xlabel('Predicted class')
            plt.ylabel('True class')
            plt.tight_layout()
            fname = os.path.join(out_folder, f"plot-confusion-{model_suffix}")
            plt.savefig(fname+'.png')
            plt.savefig(fname+'.pdf', format='pdf')
            plt.close()

    if is_ae:
        return out_folder

## Standard models

Keras models for simple or standard architectures.

### Fully connected

In [0]:
def TwoDense_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'TwoDense'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name += '-do{}'.format(dropout_rate)
    
    model = Sequential(name=name)
    model.add(Flatten(input_shape=input_shape))
    
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))

    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
def ThreeDense_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'ThreeDense'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name += '-do{}'.format(dropout_rate)
    
    model = Sequential(name=name)
    model.add(Flatten(input_shape=input_shape))
    
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
def FiveDense_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'FiveDense'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name += '-do{}'.format(dropout_rate)
    
    model = Sequential(name=name)
    model.add(Flatten(input_shape=input_shape))
    
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None))
    if dropout_rate: model.add(Dropout(dropout_rate))
    
    model.add(Dense(num_classes, activation='softmax'))
    return model

### Convolutional

In [0]:
def Conv1D_1C1D_model(input_shape, num_classes, l2_reg=None):
    name = 'Conv1D-1C1D'
    if l2_reg: name += '-reg{}'.format(l2_reg)

    return Sequential([
        Conv1D(64, 5, input_shape=input_shape, 
               kernel_regularizer=l2(l2_reg) if l2_reg else None), # shape == (batch, steps, channels)
        BatchNormalization(axis=1),
        Activation('relu'),
        MaxPooling1D(2),
        
        Flatten(),
        Dense(num_classes, activation='softmax')
    ], name=name)

In [0]:
def Conv1D_1C2D_model(input_shape, num_classes, l2_reg=None):
    name = 'Conv1D-1C2D'
    if l2_reg: name += '-reg{}'.format(l2_reg)

    return Sequential([
        Conv1D(64, 5, input_shape=input_shape,
              kernel_regularizer=l2(l2_reg) if l2_reg else None),
        BatchNormalization(axis=1),
        Activation('relu'),
        MaxPooling1D(2),
        
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=l2(l2_reg) if l2_reg else None),
        Dense(num_classes, activation='softmax')
    ], name=name)

In [0]:
# selected for report
def Conv1D_2C1D_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'Conv1D-2C1D'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name +='-do{}'.format(dropout_rate)
        
    model = Sequential(name=name)
    model.add(Conv1D(64, 5, input_shape=input_shape,
                     kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Conv1D(32, 5, kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
def Conv1D_2C2D_model(input_shape, num_classes, l2_reg=None, dropout_rate=None):
    name = 'Conv1D-2C2D'
    if l2_reg: name += '-reg{}'.format(l2_reg)
    if dropout_rate: name +='-do{}'.format(dropout_rate)
        
    model = Sequential(name=name)
    model.add(Conv1D(64, 5, input_shape=input_shape,
                     kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Conv1D(32, 5, kernel_regularizer=l2(l2_reg) if l2_reg else None))
    model.add(BatchNormalization(axis=1))
    model.add(Activation('relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(2))
        
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
def Conv1D_Chen_model(input_shape, num_classes):
    """Chen and Xue, 'A DL approach to HAR based on single accelerometer'"""
    return Sequential([
        Conv1D(18, 2, activation='relu', input_shape=input_shape), # depth, kernel
        MaxPooling1D(2), # size, strides
        
        Conv1D(36, 2, activation='relu'),
        MaxPooling1D(2),
        
        Conv1D(24, 2, activation='relu'),
        MaxPooling1D(2),
        
        Flatten(),
        Dense(num_classes, activation='softmax')
    ], name='Conv1D-Chen')

In [0]:
def Conv1D_Rueda_model(input_shape, num_classes):
    """Moya Rueda et al., 'CNN for HAR using body-worn sensors'"""
    return Sequential([
        Conv1D(64, 5, activation='relu', input_shape=input_shape),
        Conv1D(64, 5, activation='relu'),
        MaxPooling1D(2),
        
        Conv1D(64, 5, activation='relu'),
        Conv1D(64, 5, activation='relu'),
        MaxPooling1D(2),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ], name='Conv1D-Rueda')

In [0]:
def Conv2D_Bevilacqua_model(input_shape, num_classes):
    """Bevilacqua et al., 'HAR with CNNs'"""
    return Sequential([
        ZeroPadding2D((1,2), input_shape=input_shape),
        
        Conv2D(10, (3,5)), # depth kernel
        BatchNormalization(axis=2),
        Activation('relu'),
        MaxPooling2D((3,3),(1,1)), # size strides
        
        Conv2D(2, (2,4)),
        BatchNormalization(axis=2),
        Activation('relu'),
        MaxPooling2D((2,2),(1,1)),
        
        Conv2D(2, (2,2)),
        BatchNormalization(axis=2),
        Activation('relu'),
        MaxPooling2D((3,2),(1,2)),
        
        Flatten(),
        Dense(500, activation='relu'),
        Dropout(0.5),
        Dense(250, activation='relu'),
        Dropout(0.5),
        Dense(125, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ], name='Conv2D-Bevilacqua')

In [0]:
# selected for paper
def Conv2D_Ha_model(input_shape, num_classes):
    """Ha, Yun and Choi, 'Multi-modal CNN for AR'"""
    return Sequential([
        Conv2D(32, (4,4), activation='relu', input_shape=input_shape),
        MaxPooling2D((3,3),(1,1)),
        
        Conv2D(64, (5,5), activation='relu'),
        MaxPooling2D((3,3),(1,1)),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ], name='Conv2D-Ha')

### Recurrent

In [0]:
def TwoLSTM_model(input_shape, num_classes):
    return Sequential([
        LSTM(512, return_sequences=True,  stateful=False, batch_input_shape=input_shape),
        Dropout(0.2),
        LSTM(512, return_sequences=False, stateful=False),
        Dense(num_classes, activation='softmax')
    ], name='TwoLSTM')

In [0]:
def TwoGRU_model(input_shape, num_classes):
    return Sequential([
        GRU(512, input_shape=input_shape, return_sequences=True),
        Dropout(0.2), #prev commented
        GRU(512, input_shape=input_shape),
        Dense(num_classes, activation='softmax')
    ], name='TwoGRU')

## Mixed model - CNN + LSTM

Keras model made by stacking different architectures.

In [0]:
def CNN_LSTM_model(input_shape, num_classes):
    filters = 256
    LSTM_feat = 128 
    #TimeDistributed: This wrapper applies a layer to every temporal slice of an input.
    #The input should be at least 3D, and the dimension of index one will be considered to be the temporal dimension.
    return Sequential([
        TimeDistributed(Conv1D(filters=filters, kernel_size=1, activation='relu'), input_shape=input_shape),
        TimeDistributed(Conv1D(filters=filters, kernel_size=3, activation='relu')),
        TimeDistributed(Dropout(0.1)),
        TimeDistributed(MaxPooling1D(pool_size=2)),
        TimeDistributed(Flatten()),
        LSTM(LSTM_feat),
        Dense(num_classes, activation='softmax')
    ], name='CNN-LSTM')

## AutoEncoders

In [0]:
def CNN_AE_model(input_shape, num_features):
    return Sequential([
        Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=input_shape, padding='same'),
        Conv1D(filters=64,  kernel_size=1, activation='relu', padding='same'),
        Conv1D(filters=64,  kernel_size=1, activation='relu', padding='same'),
        Conv1D(filters=128, kernel_size=1, activation='relu', padding='same'),
        Conv1D(filters=num_features, kernel_size=1, activation='softmax') #TODO Try with softmax
    ], name='CNN_AE')

In [0]:
def LSTM_AE_model(input_shape, num_features):
    return Sequential([
        LSTM(128, activation='relu', input_shape=input_shape, return_sequences=True), #TODO try with 64, 32/ 128,64
        LSTM(64,  activation='relu', return_sequences=True),
        #LSTM(64, activation='relu', return_sequences=True),
        LSTM(128, activation='relu', return_sequences=True),
        #LSTM(num_features, activation='relu', return_sequences=True),
        TimeDistributed(Dense(num_features, activation='softmax')) #TODO Try with softmax
    ], name='LSTM-AE')

In [0]:
def CNN_LSTM_AE_model(input_shape, num_features):
    return Sequential([
        Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=input_shape),
        Conv1D(filters=64, kernel_size=1, activation='relu'),
        #MaxPooling1D(pool_size=1),
        LSTM(128, activation='relu', return_sequences=True),
        TimeDistributed(Dense(num_features, activation='softmax')) #TODO Try with softmax
    ], name='CNN-LSTM-AE')

## Preliminary Tests

Here each model is trained according to selected configuration on the dataset split.

The goal is to have the best configuration for each network, and to select the best models for final tests.

The configuration is a dictionary that allow to set the model's parameters and callbacks:

- `optimizer` (str): the selected optimizer for training. One of [`sgd`, `sgdm`, `adam`, `rmsprop`].
- `lr` (float): the learning rate. If omitted, standard values are used for the optimizer.
- `decay` (float): learning rate decay. Valid for all optimizers with a supplied `lr`.
- `momentum` (float): gradient momentum. Only valid for `sgdm` optimizer.
- `loss` (str): type of loss to minimize.
- `metrics_to_test` (arr of str): models to evaluate based on what metrics saved. Choose from [`best-al`, `best-aol`, `best-apr`, `best-aprol`]. If not present, all models are evaluated.

- `epochs` (int) and `batch_size` (int).
- `use_validation` (bool): whether to derive validation set from training set instead of using test set to validate training.
- `shuffle` (bool): whether to shuffle data in batches or keep the same retrieval order.

- `lr_step` (int): epochs after which the learning rate is halved. Set to 0 to disable.
- `early_stop` (int): number of epochs after which to stop training if validation loss does not decrease anymore. Set to 0 to disable.
- `tg` (bool): whether to enable Telegram notification when training finishes.

Models that contain Dense or convolutional layers may use L2 regularization with the optional parameter `l2_reg` to the model function.

Some models may apply dropout if `dropout_rate` is set, some have it enabled by default as part of the network structure.

### Fully connected

Using two dense layers leads to variable accuracy in the range [72.5%, 87.5%] with growing loss if regularization is not applied.

Regularization and dropout lower accuracy values.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = TwoDense_model(input_shape, num_classes)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 125,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = TwoDense_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = TwoDense_model(input_shape, num_classes, dropout_rate=0.5)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 60,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = TwoDense_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.5)
run_model(model, config)

Adding one dense layer, there is no substantial changes: regularization stabilizes validation loss but lowers the accuracy by 5% from 90% to 85%.

Dropout is not effective in bounding loss and has final accuracy similar to using regularization.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 50,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = ThreeDense_model(input_shape, num_classes)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 125,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = ThreeDense_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 200,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = ThreeDense_model(input_shape, num_classes, dropout_rate=0.5)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 30,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = ThreeDense_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.5)
run_model(model, config)

Using five dense layers the network is too deep for the assigned task and it does not match interesting results.

Network reaches 90% accuracy with visible overfit, and applying dropout or regularization lowers accuracy to 85 and 75% resp.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 50,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = FiveDense_model(input_shape, num_classes)
run_model(model, config)

model = FiveDense_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.5)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 300,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

model = FiveDense_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

model = FiveDense_model(input_shape, num_classes, dropout_rate=0.5)
run_model(model, config)

### Convolutional

Using 1 convolutional and 1 dense layer we obtain poor results: overfitting and growing validation loss.

Accuracy between 85 and 90%. Regularization helps but not too much.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 200,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_1C1D_model(input_shape, num_classes)
run_model(model, config)

model = Conv1D_1C1D_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

Using 1 conv. and 2 dense layers, we achieve 95% accuracy. Regularization controls loss growing but lowers accuracy to 90%.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 100,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_1C2D_model(input_shape, num_classes)
run_model(model, config)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 125,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = Conv1D_1C2D_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

Two conv. layers and one final dense layer: of the four configuration, applying only dropout is the best setup and leads to 96% accuracy.

Adding regularization is not helpful. Standard model reaches 94% accuracy with growing loss.

_Selected as one of the best CNN models for the report._

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 200,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_2C1D_model(input_shape, num_classes)
run_model(model, config)

model = Conv1D_2C1D_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

# selected for report
model = Conv1D_2C1D_model(input_shape, num_classes, dropout_rate=0.3)
run_model(model, config)

model = Conv1D_2C1D_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.3)
run_model(model, config)

In [0]:
# this contains only the selected configuration
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 400,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_2C1D_model(input_shape, num_classes, dropout_rate=0.3)
run_model(model, config)

# NO validation version
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 400,
    'batch_size': 32,
    'use_validation': False,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_2C1D_model(input_shape, num_classes, dropout_rate=0.3)
run_model(model, config)

Two conv. layers and two dense layers: same conclusions as in previous setup.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 200,
    'batch_size': 32,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_2C2D_model(input_shape, num_classes)
run_model(model, config)

model = Conv1D_2C2D_model(input_shape, num_classes, l2_reg=0.01)
run_model(model, config)

model = Conv1D_2C2D_model(input_shape, num_classes, dropout_rate=0.3)
run_model(model, config)

model = Conv1D_2C2D_model(input_shape, num_classes, l2_reg=0.01, dropout_rate=0.3)
run_model(model, config)

Model from Chen's paper: uses only accelerometer data and overfits after 150 epochs, achieving no more than 75% accuracy.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150, # tuned
    'batch_size': 1024,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

X_tr = X_train.copy()
X_tr = X_tr[:,:,:3] # only take accelerometer data
print(f'{X_train.shape} > {X_tr.shape}')

X_te = X_test.copy()
X_te = X_te[:,:,:3]
print(f'{X_test.shape} > {X_te.shape}')

input_shape = (X_tr.shape[1], X_tr.shape[2])
model = Conv1D_Chen_model(input_shape, num_classes)
run_model(model, config, x_train = X_tr, x_test = X_te)

Model from Moya-Rueda's paper: very bad performance (no more than 65% accuracy).

In [0]:
config = {
    #'optimizer': 'adam',
    'optimizer': 'rmsprop',
    'lr': 0.01,
    'decay': 0.95,
    'loss': 'categorical_crossentropy',
    'epochs': 100,
    'batch_size': 100,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 25,
    'early_stop': 0,
    'tg': False
}

input_shape = (X_train.shape[1], X_train.shape[2])
model = Conv1D_Rueda_model(input_shape, num_classes)
run_model(model, config)

Model from Bevilacqua's paper: achieves $\approx$ 86% accuracy using 2D convolutions.

In [0]:
X_tr = X_train.copy()
X_tr = np.swapaxes(X_tr,1,2)
X_tr = X_tr[...,None] # add last dimension
print(f'{X_train.shape} > {X_tr.shape}')

X_te = X_test.copy()
X_te = np.swapaxes(X_te,1,2)
X_te = X_te[...,None]
print(f'{X_test.shape} > {X_te.shape}')

input_shape = (X_train.shape[2], X_train.shape[1], 1)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150, # tuned. visible overfit if continuing
    'batch_size': 1024,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = Conv2D_Bevilacqua_model(input_shape, num_classes)
run_model(model, config, x_train = X_tr, x_test = X_te)

config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 200, # tuned. stable
    'batch_size': 1024,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = Conv2D_Bevilacqua_model(input_shape, num_classes)
run_model(model, config, x_train = X_tr, x_test = X_te)

Model from Ha's paper: achieves $\approx$ 95% accuracy padding columns to separate signals data from different sensors.

_One of the best CNN models, selected for report._ 

In [0]:
X_tr = X_train.copy()
X_tr = np.swapaxes(X_tr,1,2)
# pad three 'cols' of zeros between each sensor data
# > ax ay az 0 0 0 gx gy gz 0 0 0 mx my mz 
for index in [9,6,3]:
    for rep in range(3):
        X_tr = np.insert(X_tr, index, 0, axis=1)
X_tr = X_tr[...,None] # add dim
print(f'{X_train.shape} > {X_tr.shape}')

X_te = X_test.copy()
X_te = np.swapaxes(X_te,1,2)
for index in [9,6,3]:
    for rep in range(3):
        X_te = np.insert(X_te, index, 0, axis=1)
X_te = X_te[...,None]
print(f'{X_test.shape} > {X_te.shape}')

input_shape = (X_tr.shape[1], X_tr.shape[2],1)

# no validation version
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150, # tuned at 100. slightly overfitting but stable val. accuracy
    'batch_size': 1024,
    'use_validation': False,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': True
}
model = Conv2D_Ha_model(input_shape, num_classes)
run_model(model, config, x_train = X_tr, x_test = X_te)

# validation version
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 200, # tuned. slightly overfitting but stable val. accuracy
    'batch_size': 1024,
    'use_validation': True,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}
model = Conv2D_Ha_model(input_shape, num_classes)
run_model(model, config, x_train = X_tr, x_test = X_te)

### Recurrent

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 100,
    'batch_size': 200,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'use_validation':False,
    'tg': False
}

input_shape = (None, X_train.shape[1], X_train.shape[2])
model = TwoLSTM_model(input_shape, num_classes)
run_model(model, config)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 200,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'use_validation':False,
    'tg': False
}

input_shape = (None, X_train.shape[-1])
model = TwoGRU_model(input_shape, num_classes)
run_model(model, config)

### CNN + LSTM

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 300,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'use_validation':False,
    'tg': False
}

folds, fold_len = 4, 32 # 4*32 = 128

input_shape = (folds, fold_len, X_train.shape[-1])
model = CNN_LSTM_model(input_shape, num_classes)
run_model(model, config,
          x_train = X_train.reshape(X_train.shape[0], folds, fold_len, X_train.shape[-1]),
          x_test = X_test.reshape(X_test.shape[0], folds, fold_len, X_test.shape[-1]))

### AutoEncoders


Workflow: train an AE > create one Input(input_shape) with input_shape of AE > add encoder part of AE > predict both train and test set  > attach network


#### CNN

In [0]:
# training of the AE
config = {
    'optimizer': 'adam',
    'loss': 'mse',
    'epochs': 20,
    'batch_size': 100,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

x_tr = X_train.reshape((X_train.shape[0]*X_train.shape[1], -1))
x_te = X_test.reshape((X_test.shape[0]*X_test.shape[1], -1))
y_tr = X_train.reshape((X_train.shape[0]*X_train.shape[1], -1))
y_te = X_test.reshape((X_test.shape[0]*X_test.shape[1], -1))

x_tr = x_tr.reshape(-1,1,x_tr.shape[1])
x_te = x_te.reshape(-1,1,x_te.shape[1])
y_tr = y_tr.reshape(-1,1,y_tr.shape[1])
y_te = x_te.reshape(-1,1,y_te.shape[1])

input_shape = (x_tr.shape[1], x_tr.shape[2])
model = CNN_AE_model(input_shape, x_tr.shape[2])

folder = run_model(model, config, x_tr, y_tr, x_te, y_te, is_autoencoder=True)

In [0]:
model = load_model(f'{folder}/model-best-a.h5')

# composing encoder network: start from input layer then stack only the first layers
DL_input = Input(input_shape)
DL_model = DL_input

# keep only encoder part
for layer in model.layers[:2]:
    DL_model = layer(DL_model)
DL_model = Model(inputs=DL_input, outputs=DL_model)

# disable gradient update
for layer in DL_model.layers:
    layer.trainable = False
DL_model.summary()

# using the encoder to predict data
data_tr = DL_model.predict(x_tr, verbose = 1)
data_tr = data_tr.reshape(X_train.shape[0], X_train.shape[1], -1)
data_te = DL_model.predict(x_te, verbose = 1)
data_te = data_te.reshape(X_test.shape[0], X_test.shape[1], -1)
print(data_tr.shape)
print(data_te.shape)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 300,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'use_validation':False,
    'tg': False
}

# running CNN-LSTM model on AE's predictions
folds, fold_len = 4, 32 # 4*32 = 128

input_shape = (folds, fold_len, data_tr.shape[-1])
model = CNN_LSTM_model(input_shape, num_classes)
run_model(model, config,
          x_train = data_tr.reshape(data_tr.shape[0], folds, fold_len, data_tr.shape[-1]),
          x_test = data_te.reshape(data_te.shape[0], folds, fold_len, data_te.shape[-1]))

#### SVM


In [0]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=10000, tol = 1e-5, n_jobs = -1)
clf.fit(data_tr.reshape(X_train.shape[0],-1), Y_train_orig)
clf.score(data_te.reshape(X_test.shape[0],-1), Y_test_orig)

#### LSTM

In [0]:
# training of the AE
config = {
    'optimizer': 'adam',
    'loss': 'mse',
    'epochs': 10,
    'batch_size': 200,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

x_tr = X_train.reshape((X_train.shape[0]*X_train.shape[1], -1))
x_te = X_test.reshape((X_test.shape[0]*X_test.shape[1], -1))
y_tr = X_train.reshape((X_train.shape[0]*X_train.shape[1], -1))
y_te = X_test.reshape((X_test.shape[0]*X_test.shape[1], -1))

x_tr = x_tr.reshape(-1,1,x_tr.shape[1])
x_te = x_te.reshape(-1,1,x_te.shape[1])
y_tr = y_tr.reshape(-1,1,y_tr.shape[1])
y_te = x_te.reshape(-1,1,y_te.shape[1])

input_shape = (x_tr.shape[1], x_tr.shape[2])
model = LSTM_AE_model(input_shape, x_tr.shape[2])

folder = run_model(model, config, x_tr, y_tr, x_te, y_te, is_autoencoder=True)

In [0]:
model = load_model(f'{folder}/model-best-a.h5')

# composing encoder network: start from input layer then stack only the first layers
DL_input = Input(input_shape)
DL_model = DL_input

# keep only encoder part
for layer in model.layers[:2]:
    DL_model = layer(DL_model)
DL_model = Model(inputs=DL_input, outputs=DL_model)

# disable gradient update
for layer in DL_model.layers:
    layer.trainable = False
DL_model.summary()

# using the encoder to predict data
data_tr = DL_model.predict(x_tr, verbose = 1)
data_tr = data_tr.reshape(X_train.shape[0], X_train.shape[1], -1)
data_te = DL_model.predict(x_te, verbose = 1)
data_te = data_te.reshape(X_test.shape[0], X_test.shape[1], -1)
print(data_tr.shape)
print(data_te.shape)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 300,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'use_validation':False,
    'tg': False
}

# running CNN-LSTM model on AE's predictions
folds, fold_len = 4, 32 # 4*32 = 128

input_shape = (folds, fold_len, data_tr.shape[-1])
model = CNN_LSTM_model(input_shape, num_classes)
run_model(model, config,
          x_train = data_tr.reshape(data_tr.shape[0], folds, fold_len, data_tr.shape[-1]),
          x_test = data_te.reshape(data_te.shape[0], folds, fold_len, data_te.shape[-1]))

#### CNN-LSTM

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'mse',
    'epochs': 10,
    'batch_size': 100,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'tg': False
}

x_tr = X_train.reshape((X_train.shape[0]*X_train.shape[1], -1))
x_te = X_test.reshape((X_test.shape[0]*X_test.shape[1], -1))
y_tr = X_train.reshape((X_train.shape[0]*X_train.shape[1], -1))
y_te = X_test.reshape((X_test.shape[0]*X_test.shape[1], -1))

x_tr = x_tr.reshape(-1,1,x_tr.shape[1])
x_te = x_te.reshape(-1,1,x_te.shape[1])
y_tr = y_tr.reshape(-1,1,y_tr.shape[1])
y_te = x_te.reshape(-1,1,y_te.shape[1])

input_shape = (x_tr.shape[1], x_tr.shape[2])
model = CNN_LSTM_AE_model(input_shape, x_tr.shape[2])

folder = run_model(model, config, x_tr, y_tr, x_te, y_te, is_autoencoder=True)

In [0]:
model = load_model(f'{folder}/model-best-a.h5')

# composing encoder network: start from input layer then stack only the first layers
DL_input = Input(input_shape)
DL_model = DL_input

# keep only encoder part
for layer in model.layers[:2]:
    DL_model = layer(DL_model)
DL_model = Model(inputs=DL_input, outputs=DL_model)

# disable gradient update
for layer in DL_model.layers:
    layer.trainable = False
DL_model.summary()

# using the encoder to predict data
data_tr = DL_model.predict(x_tr, verbose = 1)
data_tr = data_tr.reshape(X_train.shape[0], X_train.shape[1], -1)
data_te = DL_model.predict(x_te, verbose = 1)
data_te = data_te.reshape(X_test.shape[0], X_test.shape[1], -1)
print(data_tr.shape)
print(data_te.shape)

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 150,
    'batch_size': 300,
    'shuffle': True,
    'lr_step': 0,
    'early_stop': 0,
    'use_validation':False,
    'tg': False
}

# running CNN-LSTM model on AE's predictions
folds, fold_len = 4, 32 # 4*32 = 128

input_shape = (folds, fold_len, data_tr.shape[-1])
model = CNN_LSTM_model(input_shape, num_classes)
run_model(model, config,
          x_train = data_tr.reshape(data_tr.shape[0], folds, fold_len, data_tr.shape[-1]),
          x_test = data_te.reshape(data_te.shape[0], folds, fold_len, data_te.shape[-1]))

## Tests of accepted networks

We have

- 12 datasets to test (three not normalized, not augmented, adasyn, handcrafted) for both sensor and body reference frames.
- 5 selected networks from previous sections: Conv1D-2C1D, Conv2D-Ha, TwoLSTM, TwoGRU, CNN-LSTM.

A for loop trains all the standard models and the mixed model on each dataset.

Another for loop trains every autoencoder and on their predictions run all the previous networks _but_ Conv2D-Ha. Also SVMs are employed here.

**WARNING**: in the report, dataset abbreviations are different due to space constraints and coherence:
- `SADANN` becomes `SADAn`
- `SAHCNN` becomes `SAHCn`
- `SNOR` becomes `SFRA` 
- `SFRA` becomes `SFRAn` 

Same for body-referenced datasets.

In [0]:
dataset_names = {
    'ARS-train-test-body-framed-aug-onlytrain-rot-per-norm.h5' : 'BAHC',   # manual aug
    'ARS-train-test-body-framed-aug-onlytrain-rot-per.h5' : 'BAHCNN',
    'ARS-train-test-body-framed-aug-onlytrain-norm.h5' : 'BADA',           # adasyn
    'ARS-train-test-body-framed-aug-onlytrain.h5' : 'BADANN',
    'ARS-train-test-body-framed-norm.h5' : 'BNOR',                         # not augmented
    'ARS-train-test-body-framed.h5' : 'BFRA',                              # not normalized  
    'ARS-train-test-sensor-framed-aug-onlytrain-rot-per-norm.h5' : 'SAHC',
    'ARS-train-test-sensor-framed-aug-onlytrain-rot-per.h5' : 'SAHCNN',    # SAHCn in report
    'ARS-train-test-sensor-framed-aug-onlytrain-norm.h5' : 'SADA',
    'ARS-train-test-sensor-framed-aug-onlytrain.h5' : 'SADANN',            # SADAn in report
    'ARS-train-test-sensor-framed-norm.h5' : 'SNOR',                       # SFRA  in report
    'ARS-train-test-sensor-framed.h5' : 'SFRA'                             # SFRAn in report
}

In [0]:
# the input datasets to these methods changes in loops
def get_Ha_datasets(X_train, X_test):
    """Dataset reshape for Ha model."""
    X_trainHa = X_train.copy()
    X_trainHa = np.swapaxes(X_trainHa,1,2)
    for index in [9,6,3]:
        for rep in range(3): # pad three 'cols' of zeros between each sensor data
            X_trainHa = np.insert(X_trainHa, index, 0, axis=1)
    X_trainHa = X_trainHa[...,None] # add dim

    X_testHa = X_test.copy()
    X_testHa = np.swapaxes(X_testHa,1,2)
    for index in [9,6,3]:
        for rep in range(3):
            X_testHa = np.insert(X_testHa, index, 0, axis=1)
    X_testHa = X_testHa[...,None]
    
    return X_trainHa, X_testHa
    
def get_CNN_LSTM_datasets(X_train, X_test):
    """Dataset reshape for CNN-LSTM model."""
    folds, fold_len = 4, 32 # 4*32 = 128
    X_train_CL = X_train.copy()
    X_train_CL = X_train_CL.reshape(X_train.shape[0], folds, fold_len, X_train.shape[-1])

    X_test_CL = X_test.copy()
    X_test_CL = X_test_CL.reshape(X_test.shape[0], folds, fold_len, X_test.shape[-1])

    return X_train_CL, X_test_CL

### Standard and mixed models

In [0]:
for dataset in dataset_names:
    # load standard dataset
    with h5py.File(f"dataset/{dataset}",'r') as h5f:
        X_train = h5f['X_train'][:] # IMU data
        X_test  = h5f['X_test'][:]  # activities
        Y_train = h5f['Y_train'][:]
        Y_test  = h5f['Y_test'][:]

    num_classes = 7
    num_data = len(X_train)
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(Y_train.shape))
    print("X_test shape:  " + str(X_test.shape))
    print("Y_test shape:  " + str(Y_test.shape))

    Y_train_orig = Y_train.copy()
    Y_test_orig  = Y_test.copy()
    Y_train = to_categorical(Y_train, num_classes=num_classes, dtype=np.uint8)
    Y_test  = to_categorical(Y_test,  num_classes=num_classes, dtype=np.uint8) 

    
    # --- dataset reshape for autoencoders
    X_train_AE = X_train.copy().reshape((X_train.shape[0]*X_train.shape[1], -1))
    X_test_AE = X_test.copy().reshape((X_test.shape[0]*X_test.shape[1], -1))
    
    X_train_AE = X_train_AE.reshape(-1,1,X_train_AE.shape[1])
    X_test_AE = X_test_AE.reshape(-1,1,X_test_AE.shape[1])
    
    Y_train_AE = X_train_AE.copy() # maybe useless
    Y_test_AE = X_test_AE.copy()
    # ---
    
    # --- other datasets reshape
    X_train_Ha, X_test_Ha = get_Ha_datasets(X_train, X_test)
    X_train_CL, X_test_CL = get_CNN_LSTM_datasets(X_train, X_test)
    folds, fold_len = 4, 32 # 4*32 = 128
    # ---
    

    tests_list = [
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 400,
                'batch_size': 32,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': Conv1D_2C1D_model( (X_train.shape[1], X_train.shape[2]), num_classes, dropout_rate=0.3)
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 150,
                'batch_size': 1024,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False  
            },
            'model': Conv2D_Ha_model( (X_train_Ha.shape[1], X_train_Ha.shape[2], 1), num_classes),
            'X_train': X_train_Ha,
            'X_test': X_test_Ha,
            'custom_ds': 'Ha'
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 100,
                'batch_size': 200,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': TwoLSTM_model( (None, X_train.shape[1], X_train.shape[2]), num_classes)
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 150,
                'batch_size': 200,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': TwoGRU_model( (None, X_train.shape[-1]), num_classes)
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 150,
                'batch_size': 300,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': CNN_LSTM_model( (folds, fold_len, X_train.shape[-1]), num_classes),
            'X_train': X_train_CL,
            'X_test': X_test_CL,
            'custom_ds': 'CL'
        }
    ]

    # the loop
    for dic in tests_list:
        clear_output()
        model = dic['model']
        print(f"Doing {model.name}")

        if 'X_train' in dic or 'X_test' in dic:
            if 'Y_train' in dic or 'Y_test' in dic:
                # model uses custom Xs and Ys
                run_model(model, dic['config'], x_train=dic['X_train'], x_test=dic['X_test'], \
                          y_train=dic['Y_train'], y_test=dic['Y_test'], y_test_orig=dic['Y_test_orig'], folder_suffix=dataset_names[dataset])
            else:
                # model uses custom Xs
                run_model(model, dic['config'], x_train=dic['X_train'], x_test=dic['X_test'], \
                          y_train=Y_train, y_test=Y_test, y_test_orig=Y_test_orig, folder_suffix=dataset_names[dataset])
        else:
            # model uses standard dataset
            run_model(model, dic['config'], x_train=X_train, x_test=X_test, \
                      y_train=Y_train, y_test=Y_test, y_test_orig=Y_test_orig, folder_suffix=dataset_names[dataset])

### AutoEncoders

In [0]:
for dataset in dataset_names:
    # load standard dataset
    with h5py.File(f"dataset/{dataset}",'r') as h5f:
        X_train = h5f['X_train'][:] # IMU data
        X_test  = h5f['X_test'][:]  # activities
        Y_train = h5f['Y_train'][:]
        Y_test  = h5f['Y_test'][:]

    num_classes = 7
    num_data = len(X_train)
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(Y_train.shape))
    print("X_test shape:  " + str(X_test.shape))
    print("Y_test shape:  " + str(Y_test.shape))

    Y_train_orig = Y_train.copy()
    Y_test_orig  = Y_test.copy()
    Y_train = to_categorical(Y_train, num_classes=num_classes, dtype=np.uint8)
    Y_test  = to_categorical(Y_test,  num_classes=num_classes, dtype=np.uint8) 

    
    # --- dataset reshape for autoencoders
    X_train_AE = X_train.copy().reshape((X_train.shape[0]*X_train.shape[1], -1))
    X_test_AE = X_test.copy().reshape((X_test.shape[0]*X_test.shape[1], -1))
    
    X_train_AE = X_train_AE.reshape(-1,1,X_train_AE.shape[1])
    X_test_AE = X_test_AE.reshape(-1,1,X_test_AE.shape[1])
    
    Y_train_AE = X_train_AE.copy() # maybe useless
    Y_test_AE = X_test_AE.copy()
    # ---
    
    # --- other datasets reshape
    X_train_CL, X_test_CL = get_CNN_LSTM_datasets(X_train, X_test)
    folds, fold_len = 4, 32 # 4*32 = 128
    # ---
    

    tests_ae_list = [ # autoencoders to train
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'mse',
                'epochs': 20,
                'batch_size': 100,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': CNN_AE_model( (X_train_AE.shape[1], X_train_AE.shape[2]), X_train_AE.shape[2]),
            'X_train': X_train_AE,
            'X_test': X_test_AE,
            'Y_train': Y_train_AE, # or maybe X directly
            'Y_test': Y_test_AE
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'mse',
                'epochs': 10,
                'batch_size': 200,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': LSTM_AE_model( (X_train_AE.shape[1], X_train_AE.shape[2]), X_train_AE.shape[2]),
            'X_train': X_train_AE,
            'X_test': X_test_AE,
            'Y_train': Y_train_AE,
            'Y_test': Y_test_AE
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'mse',
                'epochs': 10,
                'batch_size': 100,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': CNN_LSTM_AE_model( (X_train_AE.shape[1], X_train_AE.shape[2]), X_train_AE.shape[2]),
            'X_train': X_train_AE,
            'X_test': X_test_AE,
            'Y_train': Y_train_AE,
            'Y_test': Y_test_AE            
        }        
    ]

    
    tests_list_64 = [ # networks to attach to autoencoders
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 400,
                'batch_size': 32,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': Conv1D_2C1D_model( (X_train.shape[1], 64), num_classes, dropout_rate=0.3)
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 100,
                'batch_size': 200,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': TwoLSTM_model( (None, X_train.shape[1], 64), num_classes)
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 150,
                'batch_size': 200,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': TwoGRU_model( (None, 64), num_classes)
        },
        {
            'config': {
                'optimizer': 'adam',
                'loss': 'categorical_crossentropy',
                'epochs': 150,
                'batch_size': 300,
                'use_validation': False,
                'shuffle': True,
                'lr_step': 0,
                'early_stop': 0,
                'tg': False
            },
            'model': CNN_LSTM_model( (folds, fold_len, 64), num_classes),
            'X_train': X_train_CL,
            'X_test': X_test_CL,
            #'custom_ds': 'CL'
        }
    ]
    
    
    
    # the AE training loop
    folders = []
    for dic in tests_ae_list:
        clear_output()
        model = dic['model']
        print(f"Doing {model.name}")
        
        # AE training
        folder = run_model(model, dic['config'], x_train=dic['X_train'], x_test=dic['X_test'], \
                          y_train=dic['Y_train'], y_test=dic['Y_test'], is_autoencoder=True, folder_suffix=f"{dataset_names[dataset]}_train")
        folders.append(folder)
    
    
    
    # the networks part loop
    for folder in folders:    
        model_AE = load_model(f'{folder}/model-best-a.h5')

        # composing encoder network
        input_shape = (X_train_AE.shape[1], X_train_AE.shape[2])
        DL_input = Input(input_shape)
        DL_model = DL_input

        # keep only encoder part
        for layer in model_AE.layers[:2]:
            DL_model = layer(DL_model)
        DL_model = Model(inputs=DL_input, outputs=DL_model)

        # disable gradient update
        for layer in DL_model.layers:
            layer.trainable = False

        # using the encoder to predict data
        print('Predictions')
        data_tr = DL_model.predict(dic['X_train'], verbose = 1)
        X_train_AE_predicted = data_tr.reshape(X_train.shape[0], X_train.shape[1], -1)
        data_te = DL_model.predict(dic['X_test'], verbose = 1)
        X_test_AE_predicted = data_te.reshape(X_test.shape[0], X_test.shape[1], -1)
        
        # recomputing reshapes for some networks
        X_train_Ha, X_test_Ha = get_Ha_datasets(X_train_AE_predicted, X_test_AE_predicted)
        X_train_CL, X_test_CL = get_CNN_LSTM_datasets(X_train_AE_predicted, X_test_AE_predicted)
        
        # the networks training and evaluation loop
        for dic2 in tests_list_64:
            clear_output()
            model = dic2['model']
            print(f"Doing {model.name}")

            if 'X_train' in dic2 or 'X_test' in dic2:
                # model uses custom Xs
                if 'custom_ds' in dic2 and 'CL' in dic2['custom_ds']:
                    run_model(model, dic2['config'], x_train=X_train_CL, x_test=X_test_CL, \
                              y_train=Y_train, y_test=Y_test, y_test_orig=Y_test_orig, folder_suffix=f"{dataset_names[dataset]}_{modelAE.name}")
                else:
                    # model uses standard dataset
                    run_model(model, dic2['config'], x_train=X_train_AE_predicted, x_test=X_test_AE_predicted, \
                        y_train=Y_train, y_test=Y_test, y_test_orig=Y_test_orig, folder_suffix=f"{dataset_names[dataset]}_{modelAE.name}")
            else:
                # model uses standard dataset
                run_model(model, dic2['config'], x_train=X_train_AE_predicted, x_test=X_test_AE_predicted, \
                      y_train=Y_train, y_test=Y_test, y_test_orig=Y_test_orig, folder_suffix=f"{dataset_names[dataset]}_{modelAE.name}")

## Extra - LSTM test in pure Tensorflow

In [0]:
# model definition

features = 32 # number of hidden layer's features

#batch = 1500 # TODO unused vars
#n_iters = 300
#tot_iters = Y_train.shape[0] * n_iters
#disp_iter = 1000

w = {
    'h' : tf.Variable(tf.random_normal([X_train.shape[2], features])),
    'o' : tf.Variable(tf.random_normal([features, Y_train.shape[1]], mean=1.0))
}
b = {
    'h' : tf.Variable(tf.random_normal([features])),
    'o' : tf.Variable(tf.random_normal([Y_train.shape[1]]))
}

def LSTM(X, w, b):
    # input processing
    X = tf.transpose(X,[1,0,2])         # (batch_size, steps, input)
    X = tf.reshape(X, [-1, X.shape[2]]) # (steps*batch, n_initial_"features")

    X = tf.nn.relu(tf.matmul(X, w['h']) + b['h'])
    X = tf.split(X, X_train.shape[1])
    
    # model
    l_1 = tf.contrib.rnn.BasicLSTMCell(features, forget_bias=1.0, state_is_tuple=True)
    l_2 = tf.contrib.rnn.BasicLSTMCell(features, forget_bias=1.0, state_is_tuple=True)    
    lstm = tf.contrib.rnn.MultiRNNCell([l_1,l_2], state_is_tuple=True)    
    
    # output
    out, state = tf.contrib.rnn.static_rnn(lstm, X, dtype=tf.float32)
    
    return tf.matmul(out[-1], w['o']) + b['o']

In [0]:
# define a dataset object on input
ds_obj = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32), Y_train.astype(np.float32))).repeat().batch(300)
iter = ds_obj.make_one_shot_iterator()
x, y = iter.get_next()

prediction = LSTM(x, w, b)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prediction,1), tf.argmax(y,1)),tf.float32))

# losses, optimizer
lr = 0.0025
lambda_l = 0.0015

l2_norm = lambda_l * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
softmax_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction)) + l2_norm
adam = tf.train.AdamOptimizer(learning_rate=lr).minimize(softmax_cost)

# run training
test_log  = {'loss':[], 'acc':[]}
train_log = {'loss':[], 'acc':[]}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000): #epochs
        _, l, a = sess.run([adam, softmax_cost, accuracy])
        train_log['loss'].append(l)
        train_log['acc'].append(a)
        
        l,a = sess.run([softmax_cost, accuracy], feed_dict={x:X_test.astype(np.float32), y:Y_test.astype(np.float32)})
        test_log['loss'].append(l)
        test_log['acc'].append(a)
        #print("PERFORMANCE ON TEST SET: " + \
        #      "Batch Loss = {}".format(l) + \
        #      ", Accuracy = {}".format(a))
print('Reached {}'.format(max(test_log['acc'])))

# save stuff and plots
out_folder = os.path.join('output', datetime.now(pytz.timezone('Europe/Rome')).strftime('%y%m%d-%H%M%S')+'_LSTM-TF')
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

with open(os.path.join(out_folder, 'history.json'),'w') as hfile:
    json.dump({'training':train_log, 'validation':test_log}, hfile, indent=2)

plt.figure()
plt.plot(train_log['loss'], label='Training')
plt.plot( test_log['loss'], label='Validation')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.tight_layout()
fname = os.path.join(out_folder, 'plot-loss')
plt.savefig(fname+'.png')
plt.savefig(fname+'.pdf', format='pdf')
plt.close()

plt.figure()
plt.plot(train_log['acc'], label='Training')
plt.plot( test_log['acc'], label='Validation')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.tight_layout()
fname = os.path.join(out_folder, 'plot-accuracy')
plt.savefig(fname+'.png')
plt.savefig(fname+'.pdf', format='pdf')
plt.close()